In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.augmentations import CustomAugmentations

from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, Dropout, BatchNormalization

import numpy as np
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)


In [2]:
height,width=32,32
batch_size=64
num_classes=10

In [3]:
from src.kerasCNN import *

In [4]:
# commented are aleady trained models

net_configs = [
# {"kernel_sizes":[3],
# "num_filters":[32], 
# "fc_sizes":[128],},

# {"kernel_sizes":[3, 3],
# "num_filters":[32, 64], 
# "fc_sizes":[128, 64],},


# {"kernel_sizes":[3, 3, 3],
# "num_filters":[32, 64, 64], 
# "fc_sizes":[128, 64, 64],},

# {"kernel_sizes":[5],
# "num_filters":[32], 
# "fc_sizes":[128],},

# {"kernel_sizes":[5, 5],
# "num_filters":[32, 64], 
# "fc_sizes":[128, 64],},
]



In [5]:
for config in tqdm(net_configs, desc='Configurations'):
    config_str = '_'.join([f'{k}_{v}' for k,v in config.items()])

    print(f'Running config {config_str}')
    
    histories = list()
    accuracies = list()
    class_accuracies = list()
    conf_mats = list()

    for i in tqdm(range(5), desc='Iteration'):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
        '../data/CINIC10/train',
        labels='inferred',
        label_mode='int',
        class_names=None,
        color_mode='rgb',
        batch_size=64,
        image_size=(height, width),
        shuffle=True,
        seed=i,
        validation_split=0.05,
        subset='training'
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            '../data/CINIC10/valid',
            labels='inferred',
            label_mode='int',
            class_names=None,
            color_mode='rgb',
            batch_size=64,
            image_size=(height, width),
            shuffle=True,
            seed=i,
            validation_split=0.05,
            subset='validation'
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            '../data/CINIC10/test',
            labels='inferred',
            label_mode='int',
            class_names=None,
            color_mode='rgb',
            batch_size=64,
            image_size=(height, width),
            shuffle=False,
        )

        dnn_model = create_cnn(**config)

        print(f'Fitting model {i}')
        history = dnn_model.fit(train_set, validation_data=val_set, epochs=40)

        print(f'Predicting model {i}')
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f'Accuracy: {accuracy}')
        class_accuracy = conf_mat.diagonal()/conf_mat.sum(axis=1)

        print(f'Saving model {i}')
        dnn_model.save(f'../models/ClassicCNN_{config_str}_no_aug_iter{i}.h5')
        histories.append(deepcopy(history.history))
        accuracies.append(accuracy)
        class_accuracies.append(class_accuracy)
        conf_mats.append(conf_mat)

    
    # save histories
    import pickle
    with open(f'results/HISTORY_ClassicCNN_{config_str}_no_aug.pkl', 'wb') as f:
        pickle.dump(histories, f)

    # save accuracies
    with open(f'results/ACCURACY_ClassicCNN_{config_str}_no_aug.pkl', 'wb') as f:
        pickle.dump(accuracies, f)

    # save class accuracies
    with open(f'results/CLASS_ACCURACY_ClassicCNN_{config_str}_no_aug.pkl', 'wb') as f:
        pickle.dump(class_accuracies, f)

    # save confusion matrices
    with open(f'results/CONF_MAT_ClassicCNN_{config_str}_no_aug.pkl', 'wb') as f:
        pickle.dump(conf_mats, f)




Configurations:   0%|          | 0/1 [00:00<?, ?it/s]

Running config kernel_sizes_[5, 5]_num_filters_[32, 64]_fc_sizes_[128, 64]


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0
Epoch 1/40
1336/1336 [==============================] - 40s 28ms/step - loss: 1.6534 - accuracy: 0.3945 - val_loss: 1.6092 - val_accuracy: 0.4127
Epoch 2/40
1336/1336 [==============================] - 17s 13ms/step - loss: 1.3713 - accuracy: 0.5021 - val_loss: 1.4422 - val_accuracy: 0.4851
Epoch 3/40
1336/1336 [==============================] - 17s 12ms/step - loss: 1.2450 - accuracy: 0.5525 - val_loss: 1.3454 - val_accuracy: 0.5213
Epoch 4/40
1336/1336 [==============================] - 16s 12ms/step - loss: 1.1547 - accuracy: 0.5853 - val_loss: 1.3227 - val_accuracy: 0.5213
Epoch 5/40
1336/1336 [==============================] - 16s 12ms/step - loss: 1.0760 - accuracy: 0.6150 - val_loss: 1.3698 - val_accuracy: 0.5182
Epoch 6/40
1336/1336 [==============================] - 16s 

In [13]:
# from results folder import all the  where filename contains ["ACCURACY", "ClassicCNN"]
import os
import pickle
import numpy as np

files = os.listdir('results')
files = [f for f in files if 'ACCURACY' in f and 'ClassicCNN' in f]

results = dict()
for f in files:
    with open(f'results/{f}', 'rb') as f:
        results[f] = pickle.load(f)

# get the best model
best_model = None
best_accuracy = 0
best_config = None

for k, v in results.items():
    accuracy = np.mean(v)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = k.name.split('_')[1]
        best_config = k.name.split('_')[2:]

print(f'Best model is {best_model} with accuracy {best_accuracy} and config {best_config}')

Best model is ClassicCNN with accuracy 0.5859044444444444 and config ['kernel', 'sizes', '[3, 3, 3]', 'num', 'filters', '[32, 64, 64]', 'fc', 'sizes', '[128, 64, 64]', 'no', 'aug.pkl']
